# Hybrid Encryption with AES + RSA
This notebook demonstrates hybrid encryption where a message is encrypted using AES (for speed and confidentiality) and the AES key is encrypted using RSA (for secure key sharing).

In [3]:
!pip install pycryptodome


     ---------------------------------------- 1.8/1.8 MB 935.7 kB/s eta 0:00:00


In [4]:

import secrets
from Crypto.Cipher import AES
import rsa


In [5]:

def keys():
    # Key generation
    aes_key = secrets.token_bytes(32)
    public_key, private_key = rsa.newkeys(1024)
    return aes_key, public_key, private_key


In [6]:

def encrypt(message, aes_key, public_key):
    # AES encryption with GCM
    cipher_aes = AES.new(aes_key, AES.MODE_GCM)
    nonce = cipher_aes.nonce
    ciphertext, tag = cipher_aes.encrypt_and_digest(message.encode('utf-8'))

    print("\nAES Ciphertext:", ciphertext)

    # RSA encrypt AES ciphertext
    encrypted_msg = rsa.encrypt(ciphertext, public_key)
    print("\nEncrypted Message via RSA:", encrypted_msg)

    # RSA encrypt AES key
    encrypted_key = rsa.encrypt(aes_key, public_key)
    print("\nEncrypted AES Key:", encrypted_key)

    return nonce, tag, encrypted_msg, encrypted_key


In [7]:

def decrypt(nonce, tag, encrypted_msg, encrypted_key, private_key):
    # RSA decrypt AES key
    aes_key = rsa.decrypt(encrypted_key, private_key)
    print("\nDecrypted AES Key:", aes_key)

    # RSA decrypt message
    aes_ciphertext = rsa.decrypt(encrypted_msg, private_key)
    print("\nDecrypted by RSA:", aes_ciphertext)

    # AES decryption with GCM
    cipher_aes = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    decrypted_msg = cipher_aes.decrypt_and_verify(aes_ciphertext, tag).decode('utf-8')
    print("\nFinal Decrypted Message:", decrypted_msg)


In [8]:

# === Demo ===
message = input("\nEnter the message: ")
aes_key, public_key, private_key = keys()
nonce, tag, encrypted_msg, encrypted_key = encrypt(message, aes_key, public_key)
decrypt(nonce, tag, encrypted_msg, encrypted_key, private_key)



AES Ciphertext: b'-uSD\xfd\xc4W\x9f\xc7\xe1\xf7\x07\xd1N%$S\x02\x92\x9f>\xc8\x0f(\xc3\xf0'

Encrypted Message via RSA: b'\x0eo8\xae\xa5\x9c\xb9\xa4q#W#@\xaf\xf7\xa0*i\xf3\x1b\xfa\x1c?\xd9\xce\xf8\xc9\xaa\xfaCf\xec\x95\xd94\xd3\x16\xaejE\x8d\x1b\xc3\xbf\x98\xc8\xffDkr\x90l\x92\xe2\xa2\xb2\x7f\xc2\xdc!\x15&\xb5\xb8\xca\xb0\x91B0\xc9\x92\x98\xd5\xec\xf5\xe0\xd3\x8e\x0e\xd23\x88\xae\xc4\xda"\xaa1\xd0(1\xf9\xbb\xe1zQqiz\xc5\xc5\xb6\x1a\xfa\x1f\xc8\xd4\xb4*\x94\x1d?\x06\xfb\xb2\xfd-\x87V0\x0c\x1e\xec\xe1\x00\xb0B4'

Encrypted AES Key: b'\x0fl\xc4ED\xda\x93\xf6\xee\x8c\x04=$\x0cb_\xce/L\x1bk-$+R\xd6DuF\xe6z^\xda\\q`\x9c}\xca\xb5/\xf2\x13\x9f\xe4UX]\x80\xe0\xdcJ\xe7s\x13/\xbf\xac\x08\xd6j{\xd6\x85~B\x7fk\x05.\x97\x1e\xd7\x18\xa5\xdb\x06\xd1\x94F]\xa8\xe5Fc\x8dA7b\x0b\x917cn>tv\xfb\xe1\xd1oT%\x1c*H\x9cmW\xfa\x0fI\\\xc4\xf7K\xc9s\xe4?\xbf0\t\xb5\xbd\xc3,p'

Decrypted AES Key: b'K>\xe4\x86\x10\xad\xdf\xcd\xbd+\xd9\xb10\xa2\xb6\xb1\xc6a\xc5l\xaa\x9c G\xc6<\xf6\x8c6\xcbA\x1a'

Decrypted by RSA: b'